In [22]:
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

with open("results_cluster/05_cluster_label_sentences.json", "r", encoding="utf-8") as f:
    summaries = json.load(f)

labels_list = list(summaries.values())

emb_labels = model.encode(labels_list, normalize_embeddings=True, show_progress_bar=True)
np.save("results_cluster/topic_emb_labels.npy", emb_labels)


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.52it/s]


In [27]:
emb_labels.shape

(11, 768)

In [28]:
from sklearn.cluster import KMeans

n_topics = 11
kmeans = KMeans(n_clusters=n_topics, random_state=42)
topic_ids = kmeans.fit_predict(emb_labels)

df_topics = pd.DataFrame({
    "sentence": labels_list,
    "topic": topic_ids
})
df_topics.to_csv("results_cluster/topic_cluster_labels.csv", index=False)


In [33]:
topic_keywords = {}

for tid in range(n_topics):
    idx = df_topics[df_topics["topic"] == tid].index
    centroid = emb_labels[idx].mean(axis=0)

    sims = emb_labels @ centroid
    top_sentences = [labels_list[i] for i in sims.argsort()[-5:]]

    topic_keywords[tid] = top_sentences

with open("results_cluster/topic_keywords.json", "w", encoding="utf-8") as f:
    json.dump(topic_keywords, f, ensure_ascii=False, indent=2)


In [30]:
for tid, keywords in topic_keywords.items():
    print(f"Topic {tid}:")
    for kw in keywords:
        print(f" - {kw}")
    print()

Topic 0:
 - Aarhus Fremad rykker med det uafgjorte resultat op på en fjerdeplads i 2. Division Grp. 1. På lørdag venter Holbæk B&I, som kommer på besøg i næste runde. Næsby ligger efter pointdelingen fortsat på 11. pladsen i 2. Division Grp. 1. Første mulighed for at forbedre dagens remis er på lørdag, når de i næste runde møder Middelfart på hjemmebane.   Denne artikel er autogenereret på baggrund af data fra vores leverandører af fodboldstatistikker, og den er sat sammen, da resultatet af kampen forelå - læs mere om robotjournalistik her. Hvis du oplever fejl eller uregelmæssigheder i artiklen, vil vi gerne høre om det på ebwebmaster@eb.dk
 - I mere end 10 år har Skatteministeriet og Vurderingsstyrelsen arbejdet på at finde en ny måde at genere landets knap 2 mio. boligejere på. Resultatet blev offentliggjort i den forløbne uge under stor moro. Fremover skal der betales skat som 1 pct. af en opdigtet værdi, statens it-system fastsætter ved hjælp af en computer.  Det illustrerer såre 

In [32]:
df_topics

0      2
1      3
2      6
3      8
4      0
5      4
6      1
7      9
8      5
9     10
10     7
Name: topic, dtype: int32